# RHAPSODY Basic Usage Tutorial

This tutorial walks you through the fundamentals of RHAPSODY using the **`ConcurrentExecutionBackend`** with a `ProcessPoolExecutor`. No HPC cluster or Dragon runtime needed — everything runs on your local machine.

You will learn:

1. How to run 100 function tasks and collect return values
2. How to run 100 executable tasks and process stdout/stderr
3. How to detect failed tasks and resubmit them

### Prerequisites

```bash
pip install "rhapsody-py"
pip install cloudpickle   # Required for ProcessPoolExecutor
```

### Core Concepts

| Concept | What It Does |
|---|---|
| **`ComputeTask`** | A unit of work — a Python function or a shell command. |
| **`ConcurrentExecutionBackend`** | Runs tasks locally using Python's `ProcessPoolExecutor` or `ThreadPoolExecutor`. |
| **`Session`** | Connects tasks to backends, submits them, and tracks their lifecycle. |

In [ ]:
import asyncio
from concurrent.futures import ProcessPoolExecutor

from rhapsody.api import ComputeTask, Session
from rhapsody.backends import ConcurrentExecutionBackend

---

## 1. Running 100 Function Tasks

Let's start by running 100 Python functions in parallel. Each function takes an integer, does some computation, and returns a result.

After completion, every task object is updated **in-place** with:

| Field | Description |
|---|---|
| `task.state` | `"DONE"` or `"FAILED"` |
| `task.return_value` | The function's return value |
| `task.exit_code` | `0` on success, `1` on failure |
| `task.exception` | The exception object if the task failed |

In [ ]:
def compute_square(n):
    """Compute the square of a number."""
    return {"input": n, "result": n * n}

In [ ]:
async def run_100_functions():
    # Create a backend with a ProcessPoolExecutor (4 workers)
    backend = ConcurrentExecutionBackend(
        executor=ProcessPoolExecutor(max_workers=4)
    )
    session = Session(backends=[backend])

    # Create 100 function tasks
    tasks = [
        ComputeTask(function=compute_square, args=(i,))
        for i in range(100)
    ]

    async with session:
        await session.submit_tasks(tasks)
        await session.wait_tasks(tasks)

        # Count results
        done = [t for t in tasks if t.state == "DONE"]
        failed = [t for t in tasks if t.state == "FAILED"]
        print(f"Done: {len(done)}, Failed: {len(failed)}")

        # Show first 5 results
        for t in tasks[:5]:
            print(f"  {t.uid} -> {t.return_value}")

        # Verify all results
        for t in done:
            n = t.return_value["input"]
            assert t.return_value["result"] == n * n
        print("All 100 results verified.")

await run_100_functions()

**Expected output:**
```
Done: 100, Failed: 0
  task.000001 -> {'input': 0, 'result': 0}
  task.000002 -> {'input': 1, 'result': 1}
  task.000003 -> {'input': 2, 'result': 4}
  task.000004 -> {'input': 3, 'result': 9}
  task.000005 -> {'input': 4, 'result': 16}
All 100 results verified.
```

---

## 2. Running 100 Executable Tasks

Now let's run 100 shell commands. For executable tasks, results are captured as:

| Field | Description |
|---|---|
| `task.stdout` | Standard output (string) |
| `task.stderr` | Standard error (string) |
| `task.exit_code` | Process exit code (`0` = success) |
| `task.state` | `"DONE"` if exit code is 0, `"FAILED"` otherwise |

In [ ]:
async def run_100_executables():
    backend = ConcurrentExecutionBackend(
        executor=ProcessPoolExecutor(max_workers=4)
    )
    session = Session(backends=[backend])

    # Create 100 executable tasks: each echoes its index
    tasks = [
        ComputeTask(
            executable="/bin/echo",
            arguments=[f"Task {i}: result={i * i}"],
        )
        for i in range(100)
    ]

    async with session:
        await session.submit_tasks(tasks)
        await session.wait_tasks(tasks)

        done = [t for t in tasks if t.state == "DONE"]
        failed = [t for t in tasks if t.state == "FAILED"]
        print(f"Done: {len(done)}, Failed: {len(failed)}")

        # Show first 5 results
        for t in tasks[:5]:
            print(f"  {t.uid} | exit_code={t.exit_code} | stdout={t.stdout.strip()!r}")

await run_100_executables()

**Expected output:**
```
Done: 100, Failed: 0
  task.000001 | exit_code=0 | stdout='Task 0: result=0'
  task.000002 | exit_code=0 | stdout='Task 1: result=1'
  task.000003 | exit_code=0 | stdout='Task 2: result=4'
  task.000004 | exit_code=0 | stdout='Task 3: result=9'
  task.000005 | exit_code=0 | stdout='Task 4: result=16'
```

### Processing stdout and stderr

Let's run a command that writes to both stdout and stderr, and parse the output.

In [ ]:
async def process_stdout_stderr():
    backend = ConcurrentExecutionBackend(
        executor=ProcessPoolExecutor(max_workers=4)
    )
    session = Session(backends=[backend])

    # A command that writes to both stdout and stderr
    tasks = [
        ComputeTask(
            executable="/bin/bash",
            arguments=[
                "-c",
                f"echo 'stdout line {i}' && echo 'stderr warning {i}' >&2",
            ],
        )
        for i in range(5)
    ]

    async with session:
        await session.submit_tasks(tasks)
        await session.wait_tasks(tasks)

        for t in tasks:
            print(f"{t.uid}:")
            print(f"  stdout : {t.stdout.strip()!r}")
            print(f"  stderr : {t.stderr.strip()!r}")
            print(f"  exit   : {t.exit_code}")
            print()

await process_stdout_stderr()

### Parsing structured output

A common pattern is to have your command output JSON, then parse it from `task.stdout`.

In [ ]:
import json


async def parse_json_output():
    backend = ConcurrentExecutionBackend(
        executor=ProcessPoolExecutor(max_workers=4)
    )
    session = Session(backends=[backend])

    # Commands that output JSON
    tasks = [
        ComputeTask(
            executable="/bin/bash",
            arguments=[
                "-c",
                f'echo \'{{"id": {i}, "value": {i * 10}}}\'',
            ],
        )
        for i in range(5)
    ]

    async with session:
        await session.submit_tasks(tasks)
        await session.wait_tasks(tasks)

        # Parse JSON from stdout
        for t in tasks:
            data = json.loads(t.stdout.strip())
            print(f"  {t.uid} -> id={data['id']}, value={data['value']}")

await parse_json_output()

---

## 3. Error Handling and Task Resubmission

Tasks can fail for many reasons — a function raises an exception, a command returns a non-zero exit code, or a timeout expires. RHAPSODY captures all of this on the task object so you can inspect the failure and decide what to do.

### Detecting Failed Tasks

After `wait_tasks()`, check each task's `state`:

- `"DONE"` — task completed successfully
- `"FAILED"` — task failed (check `task.exception` or `task.stderr`)

In [ ]:
def sometimes_fails(n):
    """A function that fails for even numbers."""
    if n % 2 == 0:
        raise ValueError(f"Task {n} failed: even numbers not allowed!")
    return {"input": n, "result": n * n}


async def detect_failures():
    backend = ConcurrentExecutionBackend(
        executor=ProcessPoolExecutor(max_workers=4)
    )
    session = Session(backends=[backend])

    tasks = [
        ComputeTask(function=sometimes_fails, args=(i,))
        for i in range(10)
    ]

    async with session:
        await session.submit_tasks(tasks)
        await session.wait_tasks(tasks)

        for t in tasks:
            if t.state == "DONE":
                print(f"  {t.uid} OK    -> {t.return_value}")
            else:
                print(f"  {t.uid} FAIL  -> {t.stderr}")

        done = [t for t in tasks if t.state == "DONE"]
        failed = [t for t in tasks if t.state == "FAILED"]
        print(f"\nSummary: {len(done)} done, {len(failed)} failed")

await detect_failures()

### Resubmitting Failed Tasks

A practical pattern: run a batch, collect failures, create new tasks with a fix, and resubmit. Since task UIDs are auto-generated and unique, you simply create fresh `ComputeTask` objects for the retry.

In the example below, tasks fail randomly. We catch the failures, create new tasks for the same inputs, and resubmit until everything succeeds.

In [ ]:
import random


def flaky_function(n):
    """A function that fails randomly ~30% of the time."""
    if random.random() < 0.3:
        raise RuntimeError(f"Random failure for input {n}")
    return {"input": n, "result": n * 2}


async def retry_failed_tasks():
    backend = ConcurrentExecutionBackend(
        executor=ProcessPoolExecutor(max_workers=4)
    )
    session = Session(backends=[backend])

    # Initial batch: 20 tasks
    pending_inputs = list(range(20))
    all_results = {}  # input -> return_value
    max_retries = 5
    attempt = 0

    async with session:
        while pending_inputs and attempt < max_retries:
            attempt += 1
            print(f"\n--- Attempt {attempt}: submitting {len(pending_inputs)} tasks ---")

            # Create tasks for remaining inputs
            tasks = [
                ComputeTask(function=flaky_function, args=(n,))
                for n in pending_inputs
            ]

            await session.submit_tasks(tasks)
            await session.wait_tasks(tasks)

            # Separate successes and failures
            still_pending = []
            for task, input_val in zip(tasks, pending_inputs):
                if task.state == "DONE":
                    all_results[input_val] = task.return_value
                else:
                    print(f"  {task.uid} failed: {task.stderr}")
                    still_pending.append(input_val)

            pending_inputs = still_pending
            print(f"  Done: {len(all_results)}, Remaining: {len(pending_inputs)}")

    if pending_inputs:
        print(f"\nGave up on {len(pending_inputs)} tasks after {max_retries} attempts.")
    else:
        print(f"\nAll 20 tasks completed in {attempt} attempt(s).")

    # Verify results
    for n, result in sorted(all_results.items()):
        assert result["result"] == n * 2
    print(f"Verified {len(all_results)} results.")

await retry_failed_tasks()

**Expected output (varies due to randomness):**
```
--- Attempt 1: submitting 20 tasks ---
  task.000003 failed: Random failure for input 2
  task.000008 failed: Random failure for input 7
  task.000015 failed: Random failure for input 14
  Done: 17, Remaining: 3

--- Attempt 2: submitting 3 tasks ---
  task.000022 failed: Random failure for input 7
  Done: 19, Remaining: 1

--- Attempt 3: submitting 1 tasks ---
  Done: 20, Remaining: 0

All 20 tasks completed in 3 attempt(s).
Verified 20 results.
```

### Resubmitting Failed Executable Tasks

The same pattern works for executable tasks. Here we use commands that fail based on exit codes.

In [ ]:
async def retry_failed_executables():
    backend = ConcurrentExecutionBackend(
        executor=ProcessPoolExecutor(max_workers=4)
    )
    session = Session(backends=[backend])

    # Mix of commands: some succeed (/bin/true), some fail (/bin/false)
    inputs = list(range(10))
    results = {}

    async with session:
        # First attempt: even-numbered tasks will use /bin/false (fail)
        tasks = [
            ComputeTask(
                executable="/bin/bash",
                arguments=[
                    "-c",
                    f"if [ $(( {i} % 2 )) -eq 0 ]; then exit 1; else echo 'Success {i}'; fi",
                ],
            )
            for i in inputs
        ]

        await session.submit_tasks(tasks)
        await session.wait_tasks(tasks)

        failed_inputs = []
        for task, i in zip(tasks, inputs):
            if task.state == "DONE":
                results[i] = task.stdout.strip()
            else:
                print(f"  Task for input {i} failed (exit_code={task.exit_code})")
                failed_inputs.append(i)

        print(f"\nFirst attempt: {len(results)} done, {len(failed_inputs)} failed")

        # Retry failed tasks with a fixed command (always succeeds)
        if failed_inputs:
            print(f"Retrying {len(failed_inputs)} failed tasks...")
            retry_tasks = [
                ComputeTask(
                    executable="/bin/echo",
                    arguments=[f"Retry success {i}"],
                )
                for i in failed_inputs
            ]

            await session.submit_tasks(retry_tasks)
            await session.wait_tasks(retry_tasks)

            for task, i in zip(retry_tasks, failed_inputs):
                results[i] = task.stdout.strip()

        print(f"\nFinal results ({len(results)} tasks):")
        for i in sorted(results):
            print(f"  Input {i}: {results[i]!r}")

await retry_failed_executables()

---

## Quick Reference

### Creating a Backend

```python
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from rhapsody.backends import ConcurrentExecutionBackend

# ProcessPoolExecutor — runs tasks in separate processes (true parallelism)
backend = ConcurrentExecutionBackend(executor=ProcessPoolExecutor(max_workers=4))

# ThreadPoolExecutor — runs tasks in threads (default if no executor given)
backend = ConcurrentExecutionBackend()  # uses ThreadPoolExecutor
```

### Task Result Fields

| Field | Function Tasks | Executable Tasks |
|---|---|---|
| `task.state` | `"DONE"` / `"FAILED"` | `"DONE"` / `"FAILED"` |
| `task.return_value` | Function's return value | `None` |
| `task.stdout` | String of return value | Command stdout |
| `task.stderr` | Exception message on failure | Command stderr |
| `task.exit_code` | `0` success, `1` failure | Process exit code |
| `task.exception` | Exception object on failure | `None` |

### Error Handling Pattern

```python
await session.submit_tasks(tasks)
await session.wait_tasks(tasks)

done = [t for t in tasks if t.state == "DONE"]
failed = [t for t in tasks if t.state == "FAILED"]

# Inspect failures
for t in failed:
    print(f"{t.uid}: {t.stderr}")  # Error message
    print(f"  Exception: {t.exception}")  # Exception object (function tasks)

# Resubmit with new ComputeTask objects
retry_tasks = [ComputeTask(function=my_func, args=(original_input,)) for ...]
await session.submit_tasks(retry_tasks)
await session.wait_tasks(retry_tasks)
```